In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd
import calendar
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
from joblib import delayed, Parallel
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.manifold import TSNE
import statsmodels.api as sm

In [ ]:
# De dataset, ik kreeg niet de raw.githubusercontent gebeuren gefixt dus is gewoon via m'n laptop.

#url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/managed_portfolios_anom_d_50.csv'
#url = 'https://github.com/Sebasleen/Seminargroup/blob/63818bef83c627710fdd0fed246c51065ab98216/managed_portfolios_anom_d_55.csv'
kmeans1 = pd.read_csv('/Users/sebasleen/Downloads/managed_portfolios_anom_d_55.csv')

print(kmeans1.columns)
#Dropping the rme & re_ew columns as these are irrelevant. As well as the momentum returns and exchsw.
kmeans1.drop(columns=['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev', 'r_exchsw', 'rme', 're_ew'], inplace=True)

#Convert date to datetime and setting it as the index given that we have time-senitive data
kmeans1['date'] = pd.to_datetime(kmeans1['date'])
kmeans1.set_index('date', inplace=True)

#The NaN are set to 0 as in the original research
kmeans1.fillna(0, inplace=True)

#Making sure that all the columns are numerical that are left
numerical_columns = kmeans1.columns

#We use robustscaler because of the number of outliers (1643 on the dataset of 13719)
scaler = RobustScaler()
kmeans1[numerical_columns] = scaler.fit_transform(kmeans1[numerical_columns])

#Print the dataframe for debugging purposes and seeing that it goes well so far
print("Kmeans clustering dataframe:")
print(kmeans1)

#Dit is de code die kijkt of er outliers zijn.

#summary_stats = kmeans1.describe()
#
## Plot boxplots
#kmeans1.boxplot()
#plt.title('Boxplot of Numerical Columns')
#plt.show()
#
## Plot histograms
#kmeans1.hist()
#plt.title('Histogram of Numerical Columns')
#plt.show()
#
## Calculate Z-scores
#z_scores = (kmeans1 - kmeans1.mean()) / kmeans1.std()
#
## Define threshold for outliers (e.g., Z-score > 3)
#outlier_threshold = 3.5
#
## Find outliers
#outliers = (z_scores > outlier_threshold) | (z_scores < -outlier_threshold)
#outliers_count = outliers.sum()
#
#print("Outliers count per column:")
#print(outliers_count)
#

#Oke ik ga niet interpolaten gezien de dagen waar niet op werd getrade veelal feestdagen waren


#Given that we have missing days, we think it would be beneifical to interpolate these using spline interpolation.
#We use spline because it fits well with our time-sensitive and non-linear datapoints.
##We make sure to exclude weekends for interpolation because there are no returns then.
#all_dates = pd.date_range(start=kmeans1.index.min(), end=kmeans1.index.max())
#dates = pd.date_range(start=kmeans1.index.min(), end=kmeans1.index.max())
#weekdays = all_dates[dates.dayofweek < 5]  # Monday to Friday
#missing_dates = weekdays.difference(kmeans1.index)
#
#interpolated_df = pd.DataFrame(index=missing_dates)
#
##Perform spline interpolation for every weekday that is missing
#for column in kmeans1.columns:
#    non_missing_dates = kmeans1[column].dropna()
#    non_missing_dates = non_missing_dates.sort_index()
#    spline_interpolator = interp1d((non_missing_dates.index - datetime(1970, 1, 1)).days.values,
#                                   non_missing_dates.values, kind='cubic', fill_value='extrapolate')
#    interpolated_values = spline_interpolator((missing_dates - datetime(1970, 1, 1)).days.values)
#    interpolated_df[column] = interpolated_values

#Readding the interpolated data back into the original kmeans dataframe
#kmeans_interpolated = pd.concat([kmeans1, interpolated_df])
#kmeans_interpolated.sort_index(inplace=True)




In [ ]:
#Noge ven kijken wat zij doen met de data van factors die pas later beginnen
first_return_dates = kmeans1.apply(lambda x: x[x != 0].index[0] if any(x != 0) else None)

# Print the result
print("First return dates for each factor:")
print(first_return_dates)


Here we do a lot of testing to see whether monthly or daily data has better elbow and silhouette scores for the number of clusters.

First monthly then daily.

We can safely assume that the optimal number is 4 and the optimal data is daily.

The code is put in another file to save space and processing power.

In [ ]:
#Here clustering per month using the daily data which has not been interpolated

# Assuming optimal number of clusters
num_clusters = 4  # Adjust this as needed

# Define the threshold for considering weights close to 0
weight_threshold = 0.05

# Dictionary to store cluster assignments for each month
monthly_cluster_assignments = {}

# Iterate over each month in the DataFrame
for month, data in kmeans1.groupby(pd.Grouper(freq='M')):
    # Perform K-means clustering for the current month
    kmeans_model = KMeans(n_clusters=num_clusters, random_state=69)
    cluster_assignments = kmeans_model.fit_predict(data)
    
    # Store cluster assignments and centroids for the current month
    cluster_centers = kmeans_model.cluster_centers_
    
    # Filter factors with weights close to 0 for each cluster
    filtered_centers = []
    for centroid in cluster_centers:
        filtered_centroid = np.array([weight if abs(weight) > weight_threshold else 0 for weight in centroid])
        filtered_centers.append(filtered_centroid)
    filtered_centers = np.array(filtered_centers)
    
    monthly_cluster_assignments[month] = (cluster_assignments, filtered_centers)

# Create a list of factor numbers (indices)
factor_indices = list(range(1, len(data.columns) + 1))

# Visualize all clusters for each month
for month, (cluster_assignments, cluster_centers) in monthly_cluster_assignments.items():
    plt.figure(figsize=(10, 8))
    plt.title(f'Clusters for {month}')
    plt.xlabel('Factor Number')
    plt.ylabel('Factor Weight')
    
    # Initialize an array to store the maximum absolute weight for each factor
    max_abs_weights = np.zeros(len(factor_indices))
    
    # Initialize an array to store the cluster ID with maximum weight for each factor
    max_weight_cluster = np.zeros(len(factor_indices), dtype=int)
    
    # Determine the maximum absolute weight and corresponding cluster ID for each factor
    for factor_index in range(len(factor_indices)):
        max_abs_weight = 0
        for cluster_id in range(num_clusters):
            weight = cluster_centers[cluster_id][factor_index]
            if abs(weight) > abs(max_abs_weight):
                max_abs_weight = weight
                max_weight_cluster[factor_index] = cluster_id + 1  # Cluster ID starts from 1
        max_abs_weights[factor_index] = max_abs_weight
    
    # Plot the factors with their maximum absolute weights
    for cluster_id in range(1, num_clusters + 1):
        cluster_weights = np.where(max_weight_cluster == cluster_id, max_abs_weights, 0)
        plt.scatter(factor_indices, cluster_weights, marker='o', label=f'Cluster {cluster_id}')
    
    # Annotate each point with its factor number
    for i, txt in enumerate(factor_indices):
        for cluster_id in range(1, num_clusters + 1):
            weight = max_abs_weights[i]
            if weight != 0:  # Exclude factors with weights below the threshold
                plt.annotate(txt, (factor_indices[i], weight), textcoords="offset points", xytext=(0,10), ha='center')
    
    plt.legend()
    plt.grid(True)
    plt.show()

Dit is eigenlijk waar alles wat er in de graphs hierboven staat wordt omgezet in strings. Dus voor elke maand voor cluster geeft die aan welke factors daarin zitten en  welke weights zij hebben.

In [ ]:
def compute_monthly_cluster_factor_values(kmeans1, num_clusters=4, weight_threshold=0.05):
    # Dictionary to store cluster factor values for each month
    monthly_cluster_factor_values = {}

    # Iterate over each month in the DataFrame
    for month, (cluster_assignments, cluster_centers) in monthly_cluster_assignments.items():
        # Initialize a dictionary to store factor values for each cluster
        cluster_factor_values = {}

        # Initialize arrays to store factor indices and weights for each cluster
        factor_indices_list = [[] for _ in range(num_clusters)]
        factor_weights_list = [[] for _ in range(num_clusters)]

        # Determine the cluster with the maximum absolute weight for each factor
        max_weight_cluster = np.argmax(np.abs(cluster_centers), axis=0) + 1  # Cluster ID starts from 1
        max_abs_weights = np.max(np.abs(cluster_centers), axis=0)

        # Store factor values for each cluster
        for factor_index in range(len(factor_indices)):
            cluster_id = max_weight_cluster[factor_index]
            weight = cluster_centers[cluster_id - 1][factor_index]  # Cluster ID starts from 1
            if abs(weight) > weight_threshold:
                factor_indices_list[cluster_id - 1].append(factor_index + 1)  # Factor indices start from 1
                factor_weights_list[cluster_id - 1].append(weight)

        # Store cluster factor values for the current month
        for cluster_id in range(num_clusters):
            cluster_factor_values[cluster_id + 1] = {'factor_indices': factor_indices_list[cluster_id],
                                                     'factor_weights': factor_weights_list[cluster_id]}
        
        monthly_cluster_factor_values[month] = cluster_factor_values

    return monthly_cluster_factor_values

# Call the function to compute and store the monthly cluster factor values
monthly_cluster_factor_weights = compute_monthly_cluster_factor_values(kmeans1)


print(monthly_cluster_factor_weights)

In [ ]:
#Hier worden ze gescaled en gedemeaned. Wat je kan zien is dat de datapunten nogal veranderen.

def scale_and_demean_factors(monthly_cluster_factor_values):
    # Dictionary to store scaled and demeaned factor values for each month
    scaled_demeaned_factors = {}

    # Compute mean and standard deviation for each factor
    factor_means = {}
    factor_stddevs = {}
    for month_factors in monthly_cluster_factor_values.values():
        for cluster_factors in month_factors.values():
            for factor_index, factor_weight in zip(cluster_factors['factor_indices'], cluster_factors['factor_weights']):
                if factor_index not in factor_means:
                    factor_means[factor_index] = []
                    factor_stddevs[factor_index] = []
                factor_means[factor_index].append(factor_weight)
    
    for factor_index, values in factor_means.items():
        factor_means[factor_index] = np.mean(values)
        factor_stddevs[factor_index] = np.std(values)

    # Scale and demean factor values for each month
    for month, month_factors in monthly_cluster_factor_values.items():
        scaled_demeaned_factors[month] = {}
        for cluster_id, cluster_factors in month_factors.items():
            scaled_demeaned_factors[month][cluster_id] = {}
            for factor_index, factor_weight in zip(cluster_factors['factor_indices'], cluster_factors['factor_weights']):
                scaled_weight = (factor_weight - factor_means[factor_index]) / factor_stddevs[factor_index]
                scaled_demeaned_factors[month][cluster_id][factor_index] = scaled_weight

    return scaled_demeaned_factors

# Call the function to scale and demean factor values
scaled_demeaned_factors = scale_and_demean_factors(monthly_cluster_factor_values)

print(scaled_demeaned_factors)
